# Active Learning in Control Theory and Reinforcement Learning

This Jupyter notebook was used for the experiments in the thesis 'Active Learning in Control Theory and Reinforcement Learning'. The code is provided as I worked with it to make the experiments reproducible.

In [ ]:
# Some imports that are needed later on

import numpy as np
from numpy.linalg import eigh
from math import sqrt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import scipy
import scipy.linalg
import scipy.optimize
from scipy.optimize import minimize
from control.matlab import *  # MATLAB-like functions
import random
from datetime import datetime
from joblib import delayed, Parallel
import gym
from gym import wrappers
from gym import spaces
import json
import joblib
import shutil
import os.path as osp, time, atexit, os
import warnings
import seaborn as sns
import pandas as pd
import os

# For the PPO implementation of OpenAI, see
# https://spinningup.openai.com/en/latest/
import torch
from torch.optim import Adam
import spinup.algos.pytorch.ppo.core as core
from spinup.utils.logx import EpochLogger
from spinup.utils.mpi_pytorch import setup_pytorch_for_mpi, sync_params, mpi_avg_grads
from spinup.utils.mpi_tools import mpi_fork, mpi_avg, proc_id, mpi_statistics_scalar, num_procs
from spinup import ppo_pytorch as ppo
from spinup.algos.pytorch.ppo.ppo import PPOBuffer

# Definition of the control methods and utilities

First, some serialization and logger methods, taken from https://github.com/openai/spinningup/

In [ ]:
# Taken from https://github.com/openai/spinningup/blob/master/spinup/utils/serialization_utils.py


def convert_json(obj):
    """ Convert obj to a version which can be serialized with JSON. """
    if is_json_serializable(obj):
        return obj
    else:
        if isinstance(obj, dict):
            return {convert_json(k): convert_json(v) 
                    for k,v in obj.items()}

        elif isinstance(obj, tuple):
            return (convert_json(x) for x in obj)

        elif isinstance(obj, list):
            return [convert_json(x) for x in obj]

        elif hasattr(obj,'__name__') and not('lambda' in obj.__name__):
            return convert_json(obj.__name__)

        elif hasattr(obj,'__dict__') and obj.__dict__:
            obj_dict = {convert_json(k): convert_json(v) 
                        for k,v in obj.__dict__.items()}
            return {str(obj): obj_dict}

        return str(obj)
    
def is_json_serializable(v):
    try:
        json.dumps(v)
        return True
    except:
        return False

In [ ]:
# Taken from https://github.com/openai/spinningup/blob/master/spinup/utils/logx.py


color2num = dict(
    gray=30,
    red=31,
    green=32,
    yellow=33,
    blue=34,
    magenta=35,
    cyan=36,
    white=37,
    crimson=38
)

def colorize(string, color, bold=False, highlight=False):
    """
    Colorize a string.
    This function was originally written by John Schulman.
    """
    attr = []
    num = color2num[color]
    if highlight: num += 10
    attr.append(str(num))
    if bold: attr.append('1')
    return '\x1b[%sm%s\x1b[0m' % (';'.join(attr), string)
class Logger:
    """
    A general-purpose logger.
    Makes it easy to save diagnostics, hyperparameter configurations, the 
    state of a training run, and the trained model.
    """

    def __init__(self, output_dir=None, output_fname='progress.txt', exp_name=None):
        """
        Initialize a Logger.
        Args:
            output_dir (string): A directory for saving results to. If 
                ``None``, defaults to a temp directory of the form
                ``/tmp/experiments/somerandomnumber``.
            output_fname (string): Name for the tab-separated-value file 
                containing metrics logged throughout a training run. 
                Defaults to ``progress.txt``. 
            exp_name (string): Experiment name. If you run multiple training
                runs and give them all the same ``exp_name``, the plotter
                will know to group them. (Use case: if you run the same
                hyperparameter configuration with multiple random seeds, you
                should give them all the same ``exp_name``.)
        """
        self.output_dir = output_dir or "/tmp/experiments/%i"%int(time.time())
        if osp.exists(self.output_dir):
            print("Warning: Log dir %s already exists! Storing info there anyway."%self.output_dir)
        else:
            os.makedirs(self.output_dir)
        self.output_file = open(osp.join(self.output_dir, output_fname), 'w')
        atexit.register(self.output_file.close)
        print(colorize("Logging data to %s"%self.output_file.name, 'green', bold=True))
        self.first_row=True
        self.log_headers = []
        self.log_current_row = {}
        self.exp_name = exp_name

    def log(self, msg, color='green'):
        """Print a colorized message to stdout."""
        print(colorize(msg, color, bold=True))

    def log_tabular(self, key, val):
        """
        Log a value of some diagnostic.
        Call this only once for each diagnostic quantity, each iteration.
        After using ``log_tabular`` to store values for each diagnostic,
        make sure to call ``dump_tabular`` to write them out to file and
        stdout (otherwise they will not get saved anywhere).
        """
        if self.first_row:
            self.log_headers.append(key)
        else:
            assert key in self.log_headers, "Trying to introduce a new key %s that you didn't include in the first iteration"%key
        assert key not in self.log_current_row, "You already set %s this iteration. Maybe you forgot to call dump_tabular()"%key
        self.log_current_row[key] = val

    def save_config(self, config):
        """
        Log an experiment configuration.
        Call this once at the top of your experiment, passing in all important
        config vars as a dict. This will serialize the config to JSON, while
        handling anything which can't be serialized in a graceful way (writing
        as informative a string as possible). 
        Example use:
        .. code-block:: python
            logger = EpochLogger(**logger_kwargs)
            logger.save_config(locals())
        """
        config_json = convert_json(config)
        if self.exp_name is not None:
            config_json['exp_name'] = self.exp_name
        output = json.dumps(config_json, separators=(',',':\t'), indent=4, sort_keys=True)
        print(colorize('Saving config:\n', color='cyan', bold=True))
        #print(output)
        with open(osp.join(self.output_dir, "config.json"), 'w') as out:
            out.write(output)

    def save_state(self, state_dict, itr=None):
        """
        Saves the state of an experiment.
        To be clear: this is about saving *state*, not logging diagnostics.
        All diagnostic logging is separate from this function. This function
        will save whatever is in ``state_dict``---usually just a copy of the
        environment---and the most recent parameters for the model you 
        previously set up saving for with ``setup_tf_saver``. 
        Call with any frequency you prefer. If you only want to maintain a
        single state and overwrite it at each call with the most recent 
        version, leave ``itr=None``. If you want to keep all of the states you
        save, provide unique (increasing) values for 'itr'.
        Args:
            state_dict (dict): Dictionary containing essential elements to
                describe the current state of training.
            itr: An int, or None. Current iteration of training.
        """
        fname = 'vars.pkl' if itr is None else 'vars%d.pkl'%itr
        try:
            joblib.dump(state_dict, osp.join(self.output_dir, fname))
        except:
            self.log('Warning: could not pickle state_dict.', color='red')
        if hasattr(self, 'tf_saver_elements'):
            self._tf_simple_save(itr)
        if hasattr(self, 'pytorch_saver_elements'):
            self._pytorch_simple_save(itr)


    def dump_tabular(self):
        """
        Write all of the diagnostics from the current iteration.
        Writes both to stdout, and to the output file.
        """
        vals = []
        key_lens = [len(key) for key in self.log_headers]
        max_key_len = max(15,max(key_lens))
        keystr = '%'+'%d'%max_key_len
        fmt = "| " + keystr + "s | %15s |"
        n_slashes = 22 + max_key_len
        #print("-"*n_slashes)
        for key in self.log_headers:
            val = self.log_current_row.get(key, "")
            valstr = "%8.3g"%val if hasattr(val, "__float__") else val
            #print(fmt%(key, valstr))
            vals.append(val)
        #print("-"*n_slashes, flush=True)
        if self.output_file is not None:
            if self.first_row:
                self.output_file.write("\t".join(self.log_headers)+"\n")
            self.output_file.write("\t".join(map(str,vals))+"\n")
            self.output_file.flush()
        self.log_current_row.clear()
        self.first_row=False
        
    def close(self):
        self.output_file.close()

In [ ]:
def runningMean(x, N):
    return np.convolve(x, np.ones(N)/N, mode='same')

def deaggregate(y):
    x = np.copy(y.values)
    for i in range(len(x)-1):
        x[len(x)-i-1] -= x[len(x)-i-2]
    return x

In [ ]:
# Global vars for tracking and labeling data at load time.
exp_idx = 0
units = dict()
# False if you want to compare different random seeds in a single plot
#condition = True

def get_datasets(logdir, condition=None, exclude=False, trajectoryAmount=201, trajectoryLength=200, runningMeanSize=1):
    """
    Recursively look through logdir for output files produced by
    spinup.logx.Logger. 
    Assumes that any file "progress.txt" is a valid hit. 
    """
    global exp_idx
    global units
    datasets = []
    for root, _, files in os.walk(logdir):
        if exclude and 'n=' in root:
            continue
        if 'progress.txt' in files:
            exp_name = None
            try:
                config_path = open(os.path.join(root,'config.json'))
                config = json.load(config_path)
                if 'exp_name' in config:
                    exp_name = config['exp_name']
            except:
                print('No file named config.json')
            condition1 = condition or exp_name or 'exp'
            condition2 = condition1 + '-' + str(exp_idx)
            exp_idx += 1
            if condition1 not in units:
                units[condition1] = 0
            unit = units[condition1]
            units[condition1] += 1

            try:
                exp_data = pd.read_table(os.path.join(root,'progress.txt'))
                if (len(exp_data) < trajectoryAmount):
                    print('Did not contain every trajectory: %s'%os.path.join(root,'progress.txt'))
                    continue
            except:
                print('Could not read from %s'%os.path.join(root,'progress.txt'))
                continue
            performance = 'MSE'
            exp_data.insert(len(exp_data.columns),'Unit',unit)
            exp_data.insert(len(exp_data.columns),'Learner',condition1)
            exp_data.insert(len(exp_data.columns),'Condition2',condition2)
            exp_data.insert(len(exp_data.columns),'Performance',exp_data[performance])
            exp_data.insert(len(exp_data.columns),'Running reward',runningMean(exp_data["Reward"].to_numpy(), runningMeanSize))
            exp_data.insert(len(exp_data.columns),'Cost of trajectory', runningMean(deaggregate(exp_data["Cost"]), runningMeanSize))
            datasets.append(exp_data)
            config_path.close()
    return datasets

In [ ]:
color_palette = dict(zip(["PPO", "Random", "Optimal", "Passive", "DC max", "DC 1/n"],sns.color_palette("tab10")))

### Generates the cost, mean squared error, reward and efficiency plots
### from the data in the given directory and saves them there.
### If useEstimation, the additional plot of the estimation error
### is produced.
def generatePlots(dataDirectory, trajectoryAmount=201, useEstimation=False, runningMeanSize=1):
    data = (get_datasets(dataDirectory, exclude=False, trajectoryAmount=trajectoryAmount, runningMeanSize=runningMeanSize))
    if isinstance(data, list):
        data = pd.concat(data, ignore_index=True)
    # Change old names
    data.loc[data["Learner"] == "Optimal control", "Learner"] = "Optimal"
    data.loc[data["Learner"] == "Passive Learner", "Learner"] = "Passive"
    data.loc[data["Learner"] == "Random Control", "Learner"] = "Random"
    data.loc[data["Learner"] == "WDC 1/n", "Learner"] = "DC 1/n"
    data.loc[(data["Learner"] == "WDC MAX") | (data["Learner"] == "DC MAX"), "Learner"] = "DC max"
    data.loc[data["Learner"] == "PPO all", "Learner"] = "PPO"
    try:
        data.rename(columns = {'Decrease in MSE per Cost': 'Decrease in MSE per cost', 
                               "Running Reward": "Running reward"}, inplace=True)
    except:
        pass
    sns.set(style="darkgrid", font_scale=1.5)
    values = ["Cost", "Cost of trajectory", "Trajectory index", "MSE", "Decrease in MSE per cost", "Running reward"]
    if useEstimation:
        values += [ "Estimation" ]
    for value in values:
        plt.figure(figsize=(10, 7.5))
        plt.tight_layout()
        data2 = data.copy(deep=True)
        if (value in ["MSE", "Decrease in MSE per cost", "Estimation"]):
            data2 = data2[data2["Learner"] != "PPO"]
        if (value in ["Estimation", "MSE", "Decrease in MSE per cost"]):
            data2 = data2[data2["Learner"] != "Optimal"]
        g = sns.lineplot(data=data2,#[data["Learner"]!="Random Control"],
                         x="Trajectory number", 
                         y=value, 
                         hue="Learner", 
                         ci='sd', 
                         palette=color_palette,
                         linewidth=3,
                         estimator=getattr(np, 'median'))
        if (value == "Trajectory index"):
            g.set(ylabel="Trajectory steps")
        if (value == "Running reward" or value == "Cost of trajectory"):
            g.set_xlim(runningMeanSize, trajectoryAmount-runningMeanSize)
            g.set_ylim(auto=True)
            if (value =="Cost of trajectory"):
                g.set(yscale="log")
        else:
            if (value != "Trajectory index"):
                g.set(yscale="log")
        plt.legend(loc='best').set_draggable(True)
        plt.savefig(dataDirectory + "/%s"%value, dpi=300, bbox_inches='tight')
    # Trajectory reached plot
    counts = data.groupby(["Learner", "Unit"]).mean().loc[:, "Trajectory index"].unstack(0)
    sns.set(style="darkgrid", font_scale=1.5)
    plt.figure(figsize=(10, 7.5))
    plt.tight_layout()
    order = ["PPO", "Optimal", "DC max", "DC 1/n", "Random", "Passive"] if useEstimation else ["PPO", "DC max", "DC 1/n", "Random", "Passive"]
    g = sns.barplot(data=counts, 
                palette=color_palette, 
                order=order,
               estimator=getattr(np, 'median'),
                   ci='sd')
    g.set(ylabel="Trajectory steps")
    plt.savefig(dataDirectory + "/Trajectory reached", dpi=300, bbox_inches='tight')

Next, the least-squares estimator which will be used in identifying the system matrices in a linear dynamical system.

In [ ]:
### Least-squares estimator for given data X and Y. If there are not sufficient samples to calculate the least-squares estimator,
### an exception is raised. Otherwise, the dispersion matrix is (inefficiently computed) stored in the dispersion variable.
class LSE():
    def __init__(self, trainingData=np.array([]), trainingDataLabel=np.array([])):
        self.X = trainingData
        self.Y = trainingDataLabel
        self.XTX = trainingData.T @ trainingData
        if (self.isXTXFullRank()):
            self.dispersion = np.linalg.inv(self.XTX)
            # The matrix is symmetric, but rounding errors render it
            # slightly non-symmetric sometimes.
            for i in range(self.dispersion.shape[0]):
                for j in range(i+1, self.dispersion.shape[1]):
                    self.dispersion[i, j] = self.dispersion[j, i]
        else:
            raise Exception("The training data did not have enough examples!")
        
        
    def isXTXFullRank(self):
        if (self.XTX.shape == ()):
            return False
        return np.linalg.matrix_rank(self.XTX) == self.XTX.shape[0]
    
    def MSE(self):
        return np.trace(self.dispersion)
    
    # [A^T; B^T]
    def parameterEstimation(self):
        return self.dispersion @ self.X.T @ self.Y
    
    def addDataRow(self, x, y):
        self.X = np.vstack([self.X, x.T])
        self.Y = np.vstack([self.Y, y.T])
        self.XTX = self.XTX + x @ x.T
        self.dispersion = self.dispersion - (self.dispersion @ x @ x.T @ self.dispersion) / (1 + x.T @ self.dispersion @ x)

In [ ]:
### A common base class for the evaluated control methods.
class ControlMethod():
    # To get an estimate for the system parameters, initial data is neccessary.
# It is the same across a single test run to compare the control methods.
# trajectoryNumber is the number of the current trajectory, i.e. if there are 10 total trajectories planned
# and the experiment is in the third, this will be 3 - 1
# trajectoryIndex: Index in the current trajectory. Will not be greater than trajectoryLength.
    def __init__(self,
                 exp_name, 
                 trainingData=np.array([]), 
                 trainingDataLabel=np.array([]), 
                 exp_number=0, 
                 trajectoryLength=50):
        self.logger = Logger(exp_name = exp_name, output_dir=dataDirectory + "/%i/"%exp_number + exp_name)
        self.ResetToInitialState(trainingData, trainingDataLabel)
        self.logger.save_config(locals())
        self.trajectoryIndex = 0
        self.globalTrajectoryIndex = 0
        self.trajectoryLength = trajectoryLength
        self.initialMSE = self.parameterEstimator.MSE()
        
        
    def calculateControl(self, stateVector):
        raise Exception("You need to implement this class.")
        
    def updateData(self, state, control, nextStateVector):
        self.parameterEstimator.addDataRow(np.vstack([state, control]), nextStateVector)
        
    def addReward(self, reward):
        self.logger.log_tabular("Reward", reward)
        
    def dumpLogger(self):
        self.globalTrajectoryIndex += self.trajectoryIndex
        self.logger.log_tabular("Trajectory index", self.trajectoryIndex)
        self.logger.log_tabular("Trajectory number", self.trajectoryNumber)
        self.logger.log_tabular("Global trajectory index", self.globalTrajectoryIndex)
        self.logger.log_tabular("MSE", self.parameterEstimator.MSE())
        if (useEstimation):
            self.logger.log_tabular("Estimation", np.linalg.norm(self.parameterEstimator.parameterEstimation() - np.hstack([A, B]).T, 2))
        self.logger.log_tabular("Cost", self.cost)
        self.logger.log_tabular("Decrease in MSE per Cost", (self.initialMSE-self.parameterEstimator.MSE())/self.cost)
        self.logger.dump_tabular()
            
    # Expects the vectors to be in column form, i.e. with a shape (x, 1)
    def updateCost(self, state, control, nextStateVector):
        self.cost += np.array((state.T @ Q @ state + control.T @ R @ control)).reshape((1,1))[0, 0]
        
    def updateCostAtEndState(self, endState):
        self.cost += np.array(endState.T @ Qfinished @ endState).reshape(1,1)[0,0]
        
    def ResetToInitialState(self, trainingData, trainingDataLabel):
        self.parameterEstimator = LSE(trainingData, trainingDataLabel)
        self.cost = 0
        self.trajectoryNumber = None
        self.ResetTrajectory()
        
    def getInformationValue(self, state, control):
        v = np.vstack([state, control])
        return v.T @ self.parameterEstimator.dispersion @ self.parameterEstimator.dispersion @ v / (1 + v.T @ self.parameterEstimator.dispersion @ v)
        
    def ResetTrajectory(self):
        if (self.trajectoryNumber is None):
            self.trajectoryNumber = 0
        else:
            self.trajectoryNumber += 1
        self.trajectoryIndex = 0

In [ ]:
### The random controller samples from a centered normal distribution with covariance structure diag(variance)
class RandomControl(ControlMethod):
    def __init__(self, trainingData=np.array([]), trainingDataLabel=np.array([]), exp_number=0, trajectoryLength=50, variance=1):
        super(RandomControl, self).__init__(exp_name="Random", 
                                            trainingData=trainingData, 
                                            trainingDataLabel=trainingDataLabel, 
                                            exp_number=exp_number, 
                                            trajectoryLength=trajectoryLength)
        self.variance = variance
        
    def calculateControl(self, stateVector):
        self.trajectoryIndex += 1
        controlDimension = self.parameterEstimator.dispersion.shape[0] - stateVector.shape[0]
        # Freeze seed so the environment will behave the same way
        seed = np.random.get_state()
        np.random.seed()
        result = np.random.normal([0]*controlDimension, [self.variance]*controlDimension).reshape((controlDimension, 1))
        np.random.set_state(seed)
        return result

In [ ]:
### The LQR Control is the base class for all dynamic programming based solutions. This class is the certainty equivalence
### controller, which assumes the system matrices are correctly estimated. As there is no probing signal injected to the system,
### this is called a passive learner. The controller is approximated by the infinite-horizon, discrete time LQG controller.
class LQRControl(ControlMethod):
    def __init__(self, trainingData=np.array([]), trainingDataLabel=np.array([]), trajectoryLength=50, exp_name="Passive", exp_number=0):
        self.trajectoryLength = trajectoryLength
        super(LQRControl, self).__init__(exp_name=exp_name,
                                         trainingData=trainingData, 
                                         trainingDataLabel=trainingDataLabel, 
                                         exp_number=exp_number,
                                         trajectoryLength=trajectoryLength)
    
    def calculateControl(self, stateVector):
        self.trajectoryIndex += 1
        if (self.K_hat is None):
            self.K_hat = np.array((dare(self.A_hat, self.B_hat, Q, R))[2])
        return -self.K_hat @ stateVector

    def ResetTrajectory(self):
        super(LQRControl, self).ResetTrajectory()
        # Get current estimates
        matrixEst = self.parameterEstimator.parameterEstimation()
        self.A_hat = matrixEst[0:matrixEst.shape[1]].T
        self.B_hat = matrixEst[matrixEst.shape[1]:].T
        self.K_hat = None


In [ ]:
### The method derived in the thesis, which adds a estimator mean squared error term to the loss function. 
### Rho is an exploration factor, which is constant here.
class DispersionControlWithConstantFactor(LQRControl):
    def __init__(self, trainingData=np.array([]), 
                 trainingDataLabel=np.array([]), 
                 trajectoryLength=50, 
                 exp_number=0,
                 rho=0,
                exp_name="DC 0.5"):
        self.rho = rho
        super(DispersionControlWithConstantFactor, self).__init__(exp_name=exp_name, 
                                                trainingData=trainingData,
                                                trainingDataLabel=trainingDataLabel, 
                                                trajectoryLength=trajectoryLength,
                                                exp_number=exp_number)
        
       
    def calculateControl(self, stateVector):
        self.trajectoryIndex += 1
        if (self.K_hat3 is None):
            sigma = self.parameterEstimator.dispersion
            nx = self.A_hat.shape[0]
            nu = self.B_hat.shape[1]
            Q_new =  (1-self.rho)*Q - self.rho*(sigma[0:nx,0:nx]@sigma[0:nx,0:nx] + sigma[0:nx,nx:] @ sigma[0:nx,nx:].T)
            R_new =  (1-self.rho)*R - self.rho*(sigma[0:nx, nx:].T @ sigma[0:nx, nx:] + sigma[nx:, nx:] @ sigma[nx:, nx:])
            # To avoid precision errors, copy upper half to lower half
            for i in range(nx):
                Q_new[i:nx, i] = Q_new[i, i:nx]
            for i in range(nu):
                R_new[i:nu,i] = R_new[i,i:nu]
            P, L, K = dare(self.A_hat, 
                     self.B_hat, 
                     Q_new, 
                     R_new,  
                     S=- self.rho*(sigma[0:nx, 0:nx] @ sigma[0:nx, nx:] + sigma[0:nx, nx:] @ sigma[nx:, nx:]),
                     E=np.identity(nx))
            self.K_hat3 = np.array(K)
        return -self.K_hat3 @ stateVector
    
    def ResetTrajectory(self):
        super(DispersionControlWithConstantFactor, self).ResetTrajectory()
        self.K_hat3 = None

In [ ]:
class DispersionControlWithVaryingFactor(DispersionControlWithConstantFactor):
    def __init__(self, trainingData=np.array([]), 
                 trainingDataLabel=np.array([]), 
                 trajectoryLength=50, 
                 exp_number=0,
                 rho="max"):
        self.rhoUpdate=rho.lower()
        self.trajectoryNumber = None
        self.parameterEstimator = LSE(trainingData, trainingDataLabel)
        self.updateRho()
        super(DispersionControlWithVaryingFactor, self).__init__(exp_name="DC " + str(rho), 
                                                trainingData=trainingData,
                                                trainingDataLabel=trainingDataLabel, 
                                                trajectoryLength=trajectoryLength,
                                                exp_number=exp_number)
    
    def updateRho(self):
        self.rho = 1 / (1 + self.parameterEstimator.MSE()**2)
        if (self.rhoUpdate=="1/n"  and not (self.trajectoryNumber is None)):
            self.rho = min(self.rho, 1/(self.trajectoryNumber+1))
        
    # Overwrite to add an update to rho, as well as logging it.
    def updateData(self, state, control, nextStateVector):
        self.parameterEstimator.addDataRow(np.vstack([state, control]), nextStateVector)
        self.updateRho()
        self.logger.log_tabular("Rho", self.rho)

   

In [ ]:
### In case the system matrices are known, this calculates th optimal controller. 
class OptimalLQRControl(LQRControl):
    def __init__(self, trainingData=np.array([]), trainingDataLabel=np.array([]), trajectoryLength=50, exp_name="Optimal", exp_number=0):
        super(OptimalLQRControl, self).__init__(exp_name=exp_name,
                                         trainingData=trainingData, 
                                         trainingDataLabel=trainingDataLabel, 
                                         exp_number=exp_number,
                                         trajectoryLength=trajectoryLength)
        self.K = np.array((dare(A, B, Q, R))[2])
    
    def calculateControl(self, stateVector):
        self.trajectoryIndex += 1
        return -self.K @ stateVector

In [ ]:
# Adapted from https://spinningup.openai.com/en/latest/algorithms/ppo.html
class PPOControlAllData(ControlMethod):
    def __init__(self,
                 observation_space,
                 action_space,
                 exp_name="PPO", 
                 trainingData=np.array([]), 
                 trainingDataLabel=np.array([]), 
                 exp_number=0, 
                 trajectoryLength=50):
        super(PPOControlAllData, self).__init__(exp_name=exp_name,
                                     trainingData=trainingData, 
                                     trainingDataLabel=trainingDataLabel, 
                                     exp_number=exp_number,
                                     trajectoryLength=trajectoryLength)
        actor_critic=core.MLPActorCritic
        self.steps_per_epoch=trajectoryLength
        epochs=50
        self.gamma=0.99
        self.clip_ratio=0.2
        pi_lr=3e-4
        vf_lr=1e-3
        self.train_pi_iters=80
        self.train_v_iters=80
        self.lam=0.97
        max_ep_len=1000
        self.target_kl=0.01
        logger_kwargs=dict()
        save_freq=10
        self.nx = trainingDataLabel.shape[1]
        self.nu = trainingData.shape[1] - trainingDataLabel.shape[1]
        obs_dim = (self.nx,)
        act_dim = (self.nu,)

        self.ac = actor_critic(observation_space, action_space)
        self.buf = PPOBuffer(obs_dim, act_dim, trajectoryLength, self.gamma, self.lam)

            # Set up optimizers for policy and value function
        self.pi_optimizer = Adam(self.ac.pi.parameters(), lr=pi_lr)
        self.vf_optimizer = Adam(self.ac.v.parameters(), lr=vf_lr)
            

    # When the monitor is active, the episode needs to be played out for the video to record. 
    # This will not count towards the statistics.
    def epoch(self, env):
        o = env.reset()
        cum_r = 0
        restarts = 0
        for t in range(self.steps_per_epoch):
            a, v, logp = self.ac.step(torch.as_tensor(o, dtype=torch.float32))

            next_o, r, d, _ = env.step(a)
            cum_r += r
            if (restarts == 0):
                self.updateCost(o.reshape((self.nx,1)), a.reshape((self.nu,1)), next_o.reshape((self.nx, 1)))

            # save and log
            self.buf.store(o, a, r, v, logp)
            
            # Update obs (critical!)
            o = next_o

            terminal = d
            epoch_ended = t==self.steps_per_epoch-1

            if epoch_ended or terminal:
                # if trajectory didn't reach terminal state, bootstrap value target
                if not terminal:
                    _, v, _ = self.ac.step(torch.as_tensor(o, dtype=torch.float32))
                else:
                    v = 0
                # Reward is sampled only for the first trajectory for logging purposes
                if (restarts == 0):
                    self.updateCostAtEndState(o.reshape((self.nx, 1)))
                    self.trajectoryIndex = t+1
                    self.addReward(cum_r)
                restarts += 1
                self.buf.finish_path(v)
                o = env.reset()
        self.update()
        # All steps are played out, so set the index accordingly.
        #self.trajectoryIndex = self.steps_per_epoch
        self.dumpLogger()
            
        # Set up function for computing PPO policy loss
    def compute_loss_pi(self, data):
        obs, act, adv, logp_old = data['obs'], data['act'], data['adv'], data['logp']

        # Policy loss
        pi, logp = self.ac.pi(obs, act)
        ratio = torch.exp(logp - logp_old)
        clip_adv = torch.clamp(ratio, 1-self.clip_ratio, 1+self.clip_ratio) * adv
        loss_pi = -(torch.min(ratio * adv, clip_adv)).mean()

        # Useful extra info
        approx_kl = (logp_old - logp).mean().item()
        ent = pi.entropy().mean().item()
        clipped = ratio.gt(1+self.clip_ratio) | ratio.lt(1-self.clip_ratio)
        clipfrac = torch.as_tensor(clipped, dtype=torch.float32).mean().item()
        pi_info = dict(kl=approx_kl, ent=ent, cf=clipfrac)

        return loss_pi, pi_info

    # Set up function for computing value loss
    def compute_loss_v(self, data):
        obs, ret = data['obs'], data['ret']
        return ((self.ac.v(obs) - ret)**2).mean()
    
    def update(self):
        data = self.buf.get()

        pi_l_old, pi_info_old = self.compute_loss_pi(data)
        pi_l_old = pi_l_old.item()
        v_l_old = self.compute_loss_v(data).item()

        # Train policy with multiple steps of gradient descent
        for i in range(self.train_pi_iters):
            self.pi_optimizer.zero_grad()
            loss_pi, pi_info = self.compute_loss_pi(data)
            kl = mpi_avg(pi_info['kl'])
            if kl > 1.5 * self.target_kl:
                break
            loss_pi.backward()
            mpi_avg_grads(self.ac.pi)    # average grads across MPI processes
            self.pi_optimizer.step()

        # Value function learning
        for i in range(self.train_v_iters):
            self.vf_optimizer.zero_grad()
            loss_v = self.compute_loss_v(data)
            loss_v.backward()
            mpi_avg_grads(self.ac.v)    # average grads across MPI processes
            self.vf_optimizer.step()
            

Next, we define the base method for every experiment. The env variable is an implementation of an OpenAI environment.

In [ ]:
def randomControl(size):
    return np.random.normal([0]*size, [1]*size)

def generateInitialDataForEnv(env, stateDimension, controlDimension, trajectoryLength=50, trajectoryAmount=100):
    Y = np.zeros((trajectoryAmount,stateDimension))
    X = np.zeros((trajectoryAmount,stateDimension+controlDimension))
    initialState = np.zeros((stateDimension, 1))
    for rowIndex in range(trajectoryAmount):
        newState = env.reset()[0:stateDimension].reshape((stateDimension,1))
        for i in range(trajectoryLength):
            state = newState
            randomAction = randomControl(controlDimension)
            newState, reward, done, _ = env.step(randomAction)
            newState = newState[0:stateDimension].reshape((stateDimension, 1))
            if (done):
                break
        data = np.vstack([state, randomAction.reshape((controlDimension, 1))]).flatten()
        X[rowIndex, :] = data
        Y[rowIndex, :] = newState.flatten()
    return (X, Y)


In [ ]:
def experiment(runIndex, 
               env, 
               stateDimension, 
               controlDimension, 
               trajectoryLength=50, 
               trajectoryAmount=100, 
               initialDataSize=100,
              videoDirectory="",
              referencePoint=np.array([])):
    baseEnv = env
    X, Y = generateInitialDataForEnv(env, stateDimension, controlDimension, trajectoryLength=trajectoryLength, trajectoryAmount=initialDataSize)
    controlMethods = [
                      PPOControlAllData(env.observation_space, env.action_space, trainingData=X, trainingDataLabel=Y, trajectoryLength=trajectoryLength, exp_number=runIndex),
                      LQRControl(X, Y, trajectoryLength=trajectoryLength, exp_number=runIndex),
                      RandomControl(X, Y, trajectoryLength=trajectoryLength, exp_number=runIndex, variance=1), 
                      #DispersionControlWithConstantFactor(X, Y, trajectoryLength=trajectoryLength, exp_number=runIndex, rho=0.5),
                     DispersionControlWithVaryingFactor(X, Y, trajectoryLength=trajectoryLength, exp_number=runIndex, rho="max"),
                     DispersionControlWithVaryingFactor(X, Y, trajectoryLength=trajectoryLength, exp_number=runIndex, rho="1/n"),
    ]
    
    if (useEstimation):
        controlMethods.append( OptimalLQRControl(X, Y, trajectoryLength=trajectoryLength, exp_number=runIndex))
    i = 0
    seeds = []
    for controlMethod in controlMethods:
        for trajectory in range(trajectoryAmount):
            # Calculate a seed which is used by every control method on the same trajectory
            if (len(seeds) <= trajectory):
                np.random.seed()
                seed = np.random.get_state()
                seeds += [seed]
            np.random.set_state(seeds[trajectory])
            if (len(videoDirectory) > 0 and runIndex == 0 and (trajectory % 200 == 0) and (controlMethod.__class__.__name__ != 'PPOControlAllData')):
                env = gym.wrappers.Monitor(
                   env, './videos/' + videoDirectory + '/' +str(trajectoryLength) + "." + str(trajectoryAmount)  + "." + str(runIndex) + "/" + str(i) + '_' + type(controlMethod).__name__ + '/' + str(trajectory), force=True)
            if (controlMethod.__class__.__name__ == 'PPOControlAllData'):
                controlMethod.epoch(env)
            else:
                newState = env.reset()[0:stateDimension].reshape((stateDimension, 1))
                cumulativeReward = 0
                for step in range(trajectoryLength):
                    state = newState
                    if (referencePoint.shape == state.shape):
                        state = state - referencePoint
                    try:
                        control = controlMethod.calculateControl(state).reshape((controlDimension, 1))
                    except Exception as e:
                        # Sometimes no finite solution can be found, so we skip this experiment run.
                        print("Error during run", runIndex, e, controlMethod)
                        return
                    newState, reward, done, _ = env.step(control.reshape((controlDimension,)))
                    cumulativeReward += reward
                    newState = newState[0:stateDimension].reshape((stateDimension, 1))
                    controlMethod.updateCost(state, control, newState-referencePoint if newState.shape == referencePoint.shape else newState)
                    if (done):
                        break
                # Only the last data point is used for updating the estimator for ensuring iid data. This can be improved significantly
                # but this approach is more simple theory-wise.
                controlMethod.updateCostAtEndState(newState)
                controlMethod.updateData(state, control, newState)
                controlMethod.addReward(cumulativeReward)
                controlMethod.dumpLogger()
            controlMethod.ResetTrajectory()
            env = baseEnv
        controlMethod.logger.close()


# Data Center experiment
This section contains the code for the data center experiment. 

In [ ]:
# System matrices Ax + Bu, cost matrices x^T Q x + u^T R u
state_dimension = 3
A = np.diag([1.1]*state_dimension) + \
np.diag([0.1]*(state_dimension-1), -1) + \
np.diag([0.1]*(state_dimension-1), 1)
B = np.identity(state_dimension)
Q = np.identity(state_dimension)
Qfinished = Q * 1000
R = 100*np.identity(state_dimension)
useEstimation=True

In [ ]:
### Accepts state and control column vectors with combined dimension ((state_dimension, 1))
### Returns state of the data center after applying the state matrices and no random disturbance
def dataCenterStepDeterministic(stateVector, controlVector):
    # Sanity checks
    if (stateVector.ndim == 2 and stateVector.shape[1] != 1 or controlVector.ndim == 2 and controlVector.shape[1] != 1):
        raise Exception("During the data center step, a column vector was expected. Got a row vector instead.")
    if (stateVector.shape[0] != A.shape[0]):
        raise Exception ("Expected the state vector to have shape {}, got {} instead.".format(A.shape[0], stateVector.shape[0]))
    if (controlVector.shape[0] != B.shape[0]):
        raise Exception ("Expected the control vector to have shape {}, got {} instead.".format(B.shape[0], controlVector.shape[0]))
    return A @ stateVector.reshape((state_dimension, 1)) + B @ controlVector.reshape((state_dimension, 1))

def dataCenterStepProbabilistic(stateVector, controlVector, std=0.1):
    return dataCenterStepDeterministic(stateVector, controlVector) + np.random.normal([0]*state_dimension, [std]*state_dimension).reshape((state_dimension,1))

### Wrapper class to "implement" OpenAI interface
class DataCenterEnvironment():
    def __init__(self, dimension, variance):
        self.dim = dimension
        self.var = variance
        self.std = sqrt(variance)
        self.currentState = np.zeros((dimension,))
        high = np.array([160]*dimension)
        self.observation_space = spaces.Box(-high, high)
        self.action_space = spaces.Box(-high, high)
    
    def reset(self):
        self.currentState = dataCenterStepProbabilistic(np.zeros((self.dim,1)), np.zeros((self.dim,1)), std=self.std).reshape((self.dim,))
        return self.currentState
    
    # OpenAI returns flattened states
    def step(self, control):
        cost = self.currentState @ Q @ self.currentState + control @ R @ control
        self.currentState = dataCenterStepProbabilistic(self.currentState, control, std=self.std).reshape((self.dim,))
        done = (self.currentState >= 80).any() or  (self.currentState <= -80).any() 
        if done:
            # Terminal cost
            cost += self.currentState @ Qfinished @ self.currentState
        return (self.currentState, -cost, done, False)

### Wrapper to always generate a new environment for an experiment
def experimentDataCenter(runIndex, stateDimension, sigma, trajectoryLength=50, trajectoryAmount=100):
    env = DataCenterEnvironment(stateDimension, sigma)
    experiment(runIndex, env, stateDimension, stateDimension, trajectoryLength=trajectoryLength, trajectoryAmount=trajectoryAmount, initialDataSize=18 )

As some initial data is needed, we generate it using a random control with standard normal distribution.

In [ ]:
amountOfRuns=16
for trajectoryLength, trajectoryAmount, sigma in [(200, 10000, 0.5), (200, 10000, 1), (200, 10000, 2)]:
    dataDirectory="./data/DataCenter/n=" + str(state_dimension) + "/sigma=" + str(sigma) + "/amount=" + \
str(trajectoryLength) + "." + str(trajectoryAmount)  + "." + str(amountOfRuns) 
    try:
        Parallel(n_jobs=8, verbose=40)(delayed(experimentDataCenter)(i, state_dimension, sigma, trajectoryLength=trajectoryLength, trajectoryAmount=trajectoryAmount) for i in range(amountOfRuns))
        generatePlots(dataDirectory, trajectoryAmount=trajectoryAmount, useEstimation=True, runningMeanSize=100)
    except Exception as e:
        print("Exception during " + dataDirectory)
        print(e)

# Lunar Lander experiment

In [ ]:
from lunar import LunarLanderContinuous

In [ ]:
useEstimation = False
Q = np.identity(6)
# punish angular velocity
Q[5,5] = 10
Q = Q*10
Qfinished=Q*10

R = np.identity(2)
R[0,0] = 3

In [ ]:
def experimentLunarLander(runIndex, trajectoryLength=50, trajectoryAmount=100):
    env = LunarLanderContinuous()
    experiment(runIndex, env, 6, 2,  trajectoryLength=trajectoryLength, trajectoryAmount=trajectoryAmount, initialDataSize=100, videoDirectory="")
    env.close()

In [ ]:
amountOfRuns=16
for trajectoryLength, trajectoryAmount in [(300, 5001)]: 
    dataDirectory="./data/LunarLanderFinal/amount=" + \
str(trajectoryLength) + "." + str(trajectoryAmount)  + "." + str(amountOfRuns)
    try:
        print(dataDirectory)
        Parallel(n_jobs=8, verbose=40)(delayed(experimentLunarLander)
                           (i, trajectoryLength=trajectoryLength, trajectoryAmount=trajectoryAmount) for i in range(amountOfRuns))
        generatePlots(dataDirectory, trajectoryAmount=trajectoryAmount, runningMeanSize=250)
    except Exception as e:
        print(e)
        print("Exception during " + dataDirectory)

# Adjusted Cart Pole

In [ ]:
from cartpole import CartPoleContinuousEnv

In [ ]:
useEstimation = True
fps = 50
Q = np.identity(4)
Q[1,1] = 10
Q[3,3] = 100
Q *= 10
Qfinished = 10*Q
R = np.identity(1)
A = np.zeros((4,4))
A[0,1] = 1
A[1,2] = -0.7171
A[2,3] = 1
A[3,2] = 15.7756
A = A/fps + np.identity(4)
B = np.zeros((4,1))
B[1] = 0.9756
B[3] = -1.4634
B = B/fps

In [ ]:
def experimentCartPole(runIndex, trajectoryLength=200, trajectoryAmount=100, std=0.01):
    env = CartPoleContinuousEnv(std)
    experiment(runIndex, env, 4, 1, trajectoryLength=trajectoryLength, trajectoryAmount=trajectoryAmount, initialDataSize=40,  videoDirectory="CartPolePerturbed")
    env.close()

In [ ]:
amountOfRuns=16
for trajectoryLength, trajectoryAmount, std in [(200, 500, 0.01),(200, 500, 0.02),(200, 500, 0.005)]:
    dataDirectory="./data/CartPolePerturbedFinal/std=" + str(std) + "amount=" + \
str(trajectoryLength) + "." + str(trajectoryAmount)  + "." + str(amountOfRuns)
    try:
        print(dataDirectory)
        Parallel(n_jobs=8, verbose=40)(delayed(experimentCartPole)
                          (i, trajectoryLength=trajectoryLength, trajectoryAmount=trajectoryAmount) for i in range(amountOfRuns))
        generatePlots(dataDirectory, trajectoryAmount=trajectoryAmount, runningMeanSize=50, useEstimation=True)
    except Exception as e:
        print(e)
        print("Exception during " + dataDirectory)